# Generate pseudo-label from CT scanned images
We will detect bounding boxes of the sand grain images and then try to detect regions of interest within those boxes. The detected objects will work as pseudo-label for DL model training.

**Author:** _Ashiqur Rahman_
**Institute:** _Norther Illinois University_
**GitHub:** _@ashiqur-rony_

## Importing Libraries

In [206]:
import os, sys
import numpy as np
import h5py
from PIL import Image
from dask.array.overlap import boundaries
from holoviews.plotting.bokeh.styles import alpha
from mypy.checkexpr import annotations
from scipy.ndimage import binary_fill_holes
from skimage.filters import threshold_otsu
from skimage.color import label2rgb
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import closing, remove_small_objects, remove_small_holes
import skimage.morphology as morphology
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from skimage.morphology import closing
from skimage.segmentation import clear_border
from spyder_kernels.utils.nsview import get_object_attrs
from scipy import ndimage
from scipy.ndimage import binary_closing, binary_fill_holes, generate_binary_structure
from collections import deque
import pandas as pd

import cv2
import json

from sympy.matrices.expressions.blockmatrix import bounds

## Defining Functions

### Utility Functions

In [152]:
def _decompose_size(size, kernel_size=3):
    """Determine number of repeated iterations for a `kernel_size` kernel.

    Returns how many repeated morphology operations with an element of size
    `kernel_size` is equivalent to a morphology with a single kernel of size
    `n`.

    """
    if kernel_size % 2 != 1:
        raise ValueError("only odd length kernel_size is supported")
    return 1 + (size - kernel_size) // (kernel_size - 1)

In [153]:
def footprint_rectangle(shape, *, dtype=np.uint8, decomposition=None):
    """Generate a rectangular or hyper-rectangular footprint.

    Generates, depending on the length and dimensions requested with `shape`,
    a square, rectangle, cube, cuboid, or even higher-dimensional versions
    of these shapes.

    Parameters
    ----------
    shape : tuple[int, ...]
        The length of the footprint in each dimension. The length of the
        sequence determines the number of dimensions of the footprint.
    dtype : data-type, optional
        The data type of the footprint.
    decomposition : {None, 'separable', 'sequence'}, optional
        If None, a single array is returned. For 'sequence', a tuple of smaller
        footprints is returned. Applying this series of smaller footprints will
        give an identical result to a single, larger footprint, but often with
        better computational performance. See Notes for more details.
        With 'separable', this function uses separable 1D footprints for each
        axis. Whether 'sequence' or 'separable' is computationally faster may
        be architecture-dependent.

    Returns
    -------
    footprint : array or tuple[tuple[ndarray, int], ...]
        A footprint consisting only of ones, i.e. every pixel belongs to the
        neighborhood. When `decomposition` is None, this is just an array.
        Otherwise, this will be a tuple whose length is equal to the number of
        unique structuring elements to apply (see Examples for more detail).

    Examples
    --------
    >>> import skimage as ski
    >>> ski.morphology.footprint_rectangle((3, 5))
    array([[1, 1, 1, 1, 1],
           [1, 1, 1, 1, 1],
           [1, 1, 1, 1, 1]], dtype=uint8)

    Decomposition will return multiple footprints that combine into a simple
    footprint of the requested shape.

    >>> ski.morphology.footprint_rectangle((9, 9), decomposition="sequence")
    ((array([[1, 1, 1],
             [1, 1, 1],
             [1, 1, 1]], dtype=uint8),
      4),)

    `"sequence"` makes sure that the decomposition only returns 1D footprints.

    >>> ski.morphology.footprint_rectangle((3, 5), decomposition="separable")
    ((array([[1],
             [1],
             [1]], dtype=uint8),
      1),
     (array([[1, 1, 1, 1, 1]], dtype=uint8), 1))

    Generate a 5-dimensional hypercube with 3 samples in each dimension

    >>> ski.morphology.footprint_rectangle((3,) * 5).shape
    (3, 3, 3, 3, 3)
    """
    has_even_width = any(width % 2 == 0 for width in shape)
    if decomposition == "sequence" and has_even_width:
        decomposition = "sequence_fallback"
    
    def partial_footprint(dim, width):
        shape_ = (1,) * dim + (width,) + (1,) * (len(shape) - dim - 1)
        fp = (np.ones(shape_, dtype=dtype), 1)
        return fp

    if decomposition is None:
        footprint = np.ones(shape, dtype=dtype)

    elif decomposition in ("separable", "sequence_fallback"):
        footprint = tuple(
            partial_footprint(dim, width) for dim, width in enumerate(shape)
        )

    elif decomposition == "sequence":
        min_width = min(shape)
        sq_reps = _decompose_size(min_width, 3)
        footprint = [(np.ones((3,) * len(shape), dtype=dtype), sq_reps)]
        for dim, width in enumerate(shape):
            if width > min_width:
                nextra = width - min_width + 1
                component = partial_footprint(dim, nextra)
                footprint.append(component)
        footprint = tuple(footprint)

    else:
        raise ValueError(f"Unrecognized decomposition: {decomposition}")

    return footprint
    

### Region Growing Functions
We experimented with different types of region growing functions.

In [154]:
def find_regions_within_mask(image, mask, tolerance=0):
    """
    Find regions of interest within the mask.
    Parameters:
        :image: numpy array, the input image
        :mask: numpy array, the mask of the bounding boxes
        :tolerance: int, the tolerance for pixel value difference when checking neighbors
    Returns:
        :regions: list of lists, each list contains the coordinates of the pixels in a region
    """
    regions = []
    visited = np.zeros_like(image, dtype=bool)
    
    # Get the coordinates of the mask
    coords = np.column_stack(np.where(mask))
    
    # Get the pixel value to compare with
    pixel_value = image[coords[0][0], coords[0][1]]
    found_different_pixel = False
    # Iterate through the mask to find the first pixel that is not part of the mask
    c = 0
    while c in range(len(coords)) and not found_different_pixel:
        row, col = coords[c]
        c += 1
        pixel_value = image[row, col]
        i = 0
        j = 0
        while i in range (image.shape[0] - row) and not found_different_pixel:
            while j in range (image.shape[1] - col) and not found_different_pixel:
                if row + i < image.shape[0] and col + j < image.shape[1]:
                    if not mask[row + i, col + j] and image[row, col] != image[row + i, col + j]:
                        pixel_value = image[row + i, col + j]
                        found_different_pixel = True
                j += 1
            i += 1
                        
    within_mask = False
    queue = []
    
    # Iterate through each pixel in the image and when we found the start of a mask,
    # we will start a BFS to find all the connected pixels with the same pixel value.
    # This will help us to find the regions of interest inside the bounding boxes.
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            if mask[row, col] and not visited[row, col]:
                # We flip the within_mask flag to check if the pixel is within the mask or not
                within_mask = not within_mask
                
            current_region = []
            
            # If we are within a mask or the pixel is part of the mask border, add it to the queue.
            if within_mask or mask[row, col]:
                queue = [(row, col)]
            
            while queue:
                r, c = queue.pop(0)
                if visited[r, c]:
                    continue
                
                visited[r, c] = True
                current_region.append((r, c))
                
                # Check neighbors in a 2x2 grid around the pixel
                for dr in [-2, -1, 0, 1, 2]:
                    for dc in [-2, -1, 0, 1, 2]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if (0 <= nr < image.shape[0] and 
                            0 <= nc < image.shape[1] and 
                            not visited[nr, nc] and 
                            within_mask and
                            abs(int(image[nr, nc]) - pixel_value) <= tolerance):
                            queue.append((nr, nc))
            
            if current_region:
                regions.append(current_region)
    
    return regions

In [155]:
# For each pixel of the image we will check if it is inside the combined mask.
# If it is, we will set a flag for the pixel and start a BFS to flag all the nearby pixels with same pixel values.
# This will help us to find the regions of interest inside the bounding boxes.

def find_regions_of_interest(image, mask, tolerance=0):
    """
    Find regions of interest inside the bounding boxes.
    Parameters:
        :image: numpy array, the input image
        :mask: numpy array, the mask of the bounding boxes
        :tolerance: int, the tolerance for pixel value difference when checking neighbors
    Returns:
        :regions: list of lists, each list contains the coordinates of the pixels in a region
    """
    regions = []
    visited = np.zeros_like(image, dtype=bool)
    
    # Get the coordinates of the mask
    coords = np.column_stack(np.where(mask))
    
    # Iterate through each pixel in the image and when we found the start of a mask,
    # we will start a BFS to find all the connected pixels with the same pixel value.
    # This will help us to find the regions of interest inside the bounding boxes.
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            if mask[row, col] and not visited[row, col]:
                # To avoid the boundary, we will take the pixel value to the right until current pixel
                # and the pixel value to the right are the same.
                # Once we find that we will use current pixel value.
                
                pixel_value = image[row, col]
                found_different_pixel = False
                
                for i in range (image.shape[0] - row):
                    for j in range (image.shape[1] - col):
                        if row + i < image.shape[0] and col + j < image.shape[1]:
                            if not mask[row + i, col + j] and image[row, col] != image[row + i, col + j]:
                                pixel_value = image[row + i, col + j]
                                found_different_pixel = True
                                break
                        if found_different_pixel:
                            break
                    if not found_different_pixel:
                        break
                
                current_region = []
                queue = [(row, col)]
                
                while queue:
                    r, c = queue.pop(0)
                    if visited[r, c]:
                        continue
                    
                    visited[r, c] = True
                    current_region.append((r, c))
                    
                    # Check neighbors in a 2x2 grid around the pixel
                    for dr in [-2, -1, 0, 1, 2]:
                        for dc in [-2, -1, 0, 1, 2]:
                            if dr == 0 and dc == 0:
                                continue
                            nr, nc = r + dr, c + dc
                            if (0 <= nr < image.shape[0] and 
                                0 <= nc < image.shape[1] and 
                                not visited[nr, nc] and 
                                abs(int(image[nr, nc]) - pixel_value) <= tolerance):
                                queue.append((nr, nc))
                
                if current_region:
                    regions.append(current_region)
    
    return regions

In [156]:
def find_regions_within_boundaries(image, mask):
    """
    Finds all contiguous regions of pixels that are located *within* the
    boundaries defined by the `mask`. This version performs a geometric
    fill. The values from the `image` are not used for region segmentation
    itself, only the `mask` defines what is a boundary and what is interior.

    Improvements over the original approach:
    1.  Seed Pixel Selection: Identifies seed pixels for BFS that are
        definitively *inside* the boundary (i.e., `not mask[pixel]`)
        and adjacent to a boundary pixel.
    2.  BFS Logic: Performs a Breadth-First Search (BFS) that expands
        as long as pixels are within image bounds, *not* part of the `mask`,
        and not yet visited by any fill operation.
    3.  Region Content: Ensures that only non-mask (interior) pixels are
        included in the found regions.
    4.  Efficiency: Uses `collections.deque` for efficient queue operations in BFS.

    Parameters:
        image: numpy array, the input image. Not directly used for segmentation logic
               in this version, but its shape is implicitly assumed to match the mask's.
               Provided for API compatibility with the original function.
        mask: numpy array (boolean or 0/1), where True/1 indicates a boundary pixel.
              Shape should be (rows, cols).
    Returns:
        regions: A list of lists. Each inner list contains (row, col) tuples
                 for pixels belonging to a distinct enclosed region.
    """

    if mask.dtype != bool:
        mask = mask.astype(bool) # Ensure mask is boolean

    # Get dimensions from the mask
    rows, cols = mask.shape
    
    # visited_fill tracks non-mask pixels that have already been assigned to a region.
    visited_fill = np.zeros_like(mask, dtype=bool)
    regions = []

    # Iterate over each pixel to find boundary pixels.
    # When a boundary pixel is found, we look for an adjacent non-boundary,
    # unvisited pixel to start a new region fill.
    for r_boundary in range(rows):
        for c_boundary in range(cols):
            # Check if the current pixel is part of a boundary
            if mask[r_boundary, c_boundary]:
                # Explore its 4-connected neighbors to find a seed pixel.
                # A seed pixel must be:
                # 1. Inside the image bounds.
                # 2. NOT a boundary pixel itself (i.e., `not mask[seed_r, seed_c]`).
                # 3. Not already part of a previously filled region.
                for dr_seed, dc_seed in [(-1, 0), (1, 0), (0, -1), (0, 1)]: # 4-connectivity
                    
                    nr_potential_seed, nc_potential_seed = r_boundary + dr_seed, c_boundary + dc_seed

                    # Check bounds for the potential seed
                    if 0 <= nr_potential_seed < rows and 0 <= nc_potential_seed < cols:
                        # Check if it's an interior pixel and not yet visited
                        if not mask[nr_potential_seed, nc_potential_seed] and \
                           not visited_fill[nr_potential_seed, nc_potential_seed]:
                            
                            # Found a valid seed pixel. Start a BFS to find the entire region.
                            current_region_coords = []
                            q_fill = deque([(nr_potential_seed, nc_potential_seed)])
                            
                            # Mark the seed pixel as visited immediately to prevent re-processing.
                            visited_fill[nr_potential_seed, nc_potential_seed] = True
                            
                            while q_fill:
                                r_curr, c_curr = q_fill.popleft()
                                current_region_coords.append((r_curr, c_curr))

                                # Explore 4-connected neighbors for the fill.
                                for dr_fill, dc_fill in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                                    next_r, next_c = r_curr + dr_fill, c_curr + dc_fill

                                    # Check bounds for the neighbor
                                    if 0 <= next_r < rows and 0 <= next_c < cols:
                                        # If the neighbor is within bounds, not yet visited,
                                        # and is an interior pixel (not a boundary),
                                        # add it to the queue and mark as visited.
                                        if not visited_fill[next_r, next_c] and \
                                           not mask[next_r, next_c]:
                                            visited_fill[next_r, next_c] = True
                                            q_fill.append((next_r, next_c))
                            
                            # If the region has pixels, add it to the list of regions.
                            if current_region_coords:
                                regions.append(current_region_coords)
    return regions

In [157]:
def find_intensity_regions_within_boundaries(image, mask):
    """
    Finds contiguous regions of similar pixel intensity that are located
    *within* the boundaries defined by the `mask`.

    This version:
    1.  Identifies seed pixels that are *inside* the boundary (not mask)
        and not yet part of a found region.
    2.  For each seed, it records its pixel intensity from the `image`.
    3.  Performs a Breadth-First Search (BFS) that expands as long as
        neighboring pixels are:
        a. Within image bounds.
        b. Not part of the `mask`.
        c. Not yet visited.
        d. Have the *same intensity* as the seed pixel.
    4.  Ensures only non-mask pixels are included in the regions.

    Parameters:
        image: numpy array (2D, grayscale). Pixel intensities from this image
               are used to define region homogeneity.
        mask: numpy array (boolean or 0/1), where True/1 indicates a boundary pixel.
              Shape should be (rows, cols) and match `image.shape`.
    Returns:
        regions: A list of lists. Each inner list contains (row, col) tuples
                 for pixels belonging to a distinct intensity-based region
                 within the boundaries.
    """

    if image.ndim != 2:
        raise ValueError("Image must be 2D (grayscale).")
    if image.shape != mask.shape:
        raise ValueError("Image and mask must have the same dimensions.")

    if mask.dtype != bool:
        mask = mask.astype(bool)

    rows, cols = image.shape
    
    # visited_pixels tracks pixels already assigned to a region.
    visited_pixels = np.zeros_like(mask, dtype=bool)
    found_regions = []

    for r_boundary in range(rows):
        for c_boundary in range(cols):
            if mask[r_boundary, c_boundary]: # It's a boundary pixel
                # Check its 4-connected neighbors to find a potential seed
                for dr_seed, dc_seed in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    
                    nr_potential_seed, nc_potential_seed = r_boundary + dr_seed, c_boundary + dc_seed

                    if 0 <= nr_potential_seed < rows and 0 <= nc_potential_seed < cols:
                        # Seed must be: not a mask pixel AND not yet visited
                        if not mask[nr_potential_seed, nc_potential_seed] and \
                           not visited_pixels[nr_potential_seed, nc_potential_seed]:
                            
                            current_seed_r, current_seed_c = nr_potential_seed, nc_potential_seed
                            seed_intensity = image[current_seed_r, current_seed_c]
                            
                            current_region_coords = []
                            q_bfs = deque([(current_seed_r, current_seed_c)])
                            
                            # Mark the exact seed pixel as visited
                            visited_pixels[current_seed_r, current_seed_c] = True
                            
                            while q_bfs:
                                r_curr, c_curr = q_bfs.popleft()
                                current_region_coords.append((r_curr, c_curr))

                                for dr_bfs, dc_bfs in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                                    next_r, next_c = r_curr + dr_bfs, c_curr + dc_bfs

                                    if 0 <= next_r < rows and 0 <= next_c < cols:
                                        if not visited_pixels[next_r, next_c] and \
                                           not mask[next_r, next_c] and \
                                           image[next_r, next_c] == seed_intensity: # Key check
                                            
                                            visited_pixels[next_r, next_c] = True
                                            q_bfs.append((next_r, next_c))
                            
                            if current_region_coords:
                                found_regions.append(current_region_coords)
    return found_regions

In [158]:
def find_intensity_range_within_boundaries(image, mask, intensity_tolerance=5):
    """
    Finds contiguous regions of similar pixel intensity that are located
    *within* the boundaries defined by the `mask`, allowing for a specified
    intensity tolerance. Seed finding is optimized to start from pixels
    adjacent to boundaries.

    This version:
    1.  Identifies seed pixels by checking neighbors of `mask` (boundary) pixels.
        A seed must be *inside* the boundary (not mask) and not yet visited.
    2.  For each seed, it records its pixel intensity from the `image`.
    3.  Performs a Breadth-First Search (BFS) that expands as long as
        neighboring pixels are:
        a. Within image bounds.
        b. Not part of the `mask`.
        c. Not yet visited.
        d. Have an intensity within `intensity_tolerance` of the seed pixel's intensity.
    4.  Ensures only non-mask pixels are included in the regions.

    Parameters:
        image: numpy array (2D, grayscale). Pixel intensities from this image
               are used to define region homogeneity. Assumed to be of a
               numeric type (e.g., uint8, int16, float).
        mask: numpy array (boolean or 0/1), where True/1 indicates a boundary pixel.
              Shape should be (rows, cols) and match `image.shape`.
        intensity_tolerance: int, the maximum allowed absolute difference
                             between a pixel's intensity and the seed pixel's
                             intensity for it to be included in the region.
                             Default is 5.
    Returns:
        regions: A list of lists. Each inner list contains (row, col) tuples
                 for pixels belonging to a distinct intensity-based region
                 within the boundaries.
    """

    if image.ndim != 2:
        raise ValueError("Image must be 2D (grayscale).")
    if image.shape != mask.shape:
        raise ValueError("Image and mask must have the same dimensions.")

    if mask.dtype != bool:
        mask = mask.astype(bool)

    rows, cols = image.shape
    
    # visited_pixels tracks pixels already assigned to a region.
    visited_pixels = np.zeros_like(mask, dtype=bool)
    found_regions = []

    # Iterate through all pixels to find boundary pixels
    for r_boundary in range(rows):
        for c_boundary in range(cols):
            # If it's a boundary pixel, check its neighbors for potential seeds
            if mask[r_boundary, c_boundary]:
                # Explore 4-connected neighbors of the boundary pixel
                for dr_seed, dc_seed in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    
                    current_seed_r, current_seed_c = r_boundary + dr_seed, c_boundary + dc_seed

                    # Check if the potential seed is within bounds
                    if 0 <= current_seed_r < rows and 0 <= current_seed_c < cols:
                        # Seed must be: not a mask pixel AND not yet visited
                        if not mask[current_seed_r, current_seed_c] and \
                           not visited_pixels[current_seed_r, current_seed_c]:
                            
                            # Found a valid seed pixel.
                            # It's crucial to handle image dtype correctly for subtraction.
                            seed_intensity = int(image[current_seed_r, current_seed_c]) 
                            
                            current_region_coords = []
                            q_bfs = deque([(current_seed_r, current_seed_c)])
                            
                            # Mark the exact seed pixel as visited
                            visited_pixels[current_seed_r, current_seed_c] = True
                            
                            while q_bfs:
                                r_curr, c_curr = q_bfs.popleft()
                                current_region_coords.append((r_curr, c_curr))

                                # Explore 4-connected neighbors for BFS expansion
                                for dr_bfs, dc_bfs in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                                    next_r, next_c = r_curr + dr_bfs, c_curr + dc_bfs

                                    if 0 <= next_r < rows and 0 <= next_c < cols:
                                        # Check if neighbor is valid for region
                                        if not visited_pixels[next_r, next_c] and \
                                           not mask[next_r, next_c]:
                                            
                                            current_pixel_intensity = int(image[next_r, next_c])
                                            # Check intensity tolerance
                                            if abs(current_pixel_intensity - seed_intensity) <= intensity_tolerance:
                                                visited_pixels[next_r, next_c] = True
                                                q_bfs.append((next_r, next_c))
                            
                            if current_region_coords:
                                found_regions.append(current_region_coords)
    return found_regions

In [159]:
def fill_gapped_objects(binary_image_array: np.ndarray, 
                        closing_iterations: int = 1, 
                        connectivity: int = 2) -> np.ndarray:
    """
    Sets all pixels within white boundaries to True in a binary NumPy array,
    attempting to close gaps in boundaries first using morphological closing,
    and then performing hole filling.

    Args:
        binary_image_array: A 2D NumPy array where True represents white pixels
                            (including boundaries and object interiors if any) 
                            and False represents black pixels. The function will
                            convert it to a boolean array if it's not already.
        closing_iterations: Number of iterations for the binary closing operation.
                            A larger number of iterations can close larger gaps
                            but may also distort the shapes more or merge very
                            close objects. Start with 1 or 2 for small gaps.
        connectivity: Defines the neighborhood for morphological operations (and hole filling).
                      For a 2D image (which this function expects):
                      - 1 creates a 4-connected structuring element (diamond shape: 
                        connects to pixels directly up, down, left, right).
                      - 2 creates an 8-connected structuring element (square shape: 
                        connects to all 8 neighboring pixels).
                      Using connectivity=2 (8-connectivity) is generally more robust 
                      for closing various gap orientations and ensuring complete hole filling.

    Returns:
        A 2D NumPy array where all objects defined by the (now hopefully closed)
        white boundaries have their interiors set to True.
    """
    if not isinstance(binary_image_array, np.ndarray):
        raise TypeError("Input must be a NumPy array.")
    if binary_image_array.ndim != 2:
        raise ValueError("Input array must be 2-dimensional.")
    if connectivity not in [1, 2]: # Assuming 2D input based on ndim check
        raise ValueError("Connectivity must be 1 (for 4-way) or 2 (for 8-way) for 2D images.")

    # Ensure the input array is boolean, as required by morphological functions.
    # This handles cases where the input might be 0s and 1s instead of True/False.
    binary_image_boolean = binary_image_array.astype(bool)

    # 1. Attempt to close gaps in the boundaries using morphological closing.
    #    A structuring element defines the neighborhood for the operation.
    #    - rank = binary_image_boolean.ndim (which is 2 for a 2D image)
    #    - connectivity (1 or 2 as per function arg) determines its shape.
    structuring_element = generate_binary_structure(rank=binary_image_boolean.ndim, 
                                                    connectivity=connectivity)
    
    # Perform binary closing. More iterations close larger gaps.
    # border_value=False ensures that operations near the border treat pixels
    # outside the image as False, preventing artificial connections to the border.
    closed_image = binary_closing(binary_image_boolean, 
                                  structure=structuring_element, 
                                  iterations=closing_iterations,
                                  border_value=False)

    # 2. Fill the holes in the (hopefully) now-closed objects.
    #    Using the same structuring element (or at least same connectivity) for
    #    hole filling ensures consistency if any fine details of the boundary
    #    matter for defining the hole.
    filled_image_array = binary_fill_holes(closed_image, 
                                           structure=structuring_element)
    
    return filled_image_array

### Retrieve Region Mask

In [160]:
def get_region_mask(regions, image=None):
    if image is None:
        raise ValueError("Image must be provided to visualize regions.")
    region_mask = np.zeros_like(image, dtype=bool)
    flatten_regions = [pixel for region in regions for pixel in region]
    # For each pixel of flatten_regions, set the corresponding pixel in the region_mask to True
    for r, c in flatten_regions:
        region_mask[r, c] = True
    return region_mask

### Cleanup Mask

In [161]:
def cleanup_mask(mask, buffer_size=0, print_output=False, image=None, min_object_size=0, min_hole_size=0, try_binary_fill_holes=False):
    """
    This function takes the mask and cleanup by binary filling holes and then cleaning boundaries.
    Parameters:
        mask: numpy array, the input mask
        buffer_size: int, the size of the buffer to clear the border
        print_output: bool, whether to print the output or not
        image: numpy array, the input image to overlay the mask on (optional)
        min_object_size: int, minimum size of objects to keep
        min_hole_size: int, minimum size of holes to fill
        try_binary_fill_holes: bool, whether to try binary filling holes or not
    Returns:
        cleared: numpy array, the cleaned mask after filling holes and clearing borders
    """
    # Binary fill holes in the mask to fill any holes inside the regions
    if try_binary_fill_holes:
        filled_mask = ndimage.binary_fill_holes(mask).astype(bool)
    else:
        filled_mask = mask
    cleared = clear_border(filled_mask, buffer_size=buffer_size)
    cleared = remove_small_objects(cleared, min_object_size)
    cleared = remove_small_holes(cleared, min_hole_size)

    if print_output:
        print(f"Filled mask shape: {filled_mask.shape}, Data type: {filled_mask.dtype}")
        cmap = mcolors.ListedColormap([(0, 0, 0, 0), (1, 1, 1, 1)])  # Transparent black, opaque white
        fig, ax = plt.subplots(figsize=(10, 10))
        if image is not None:
            ax.imshow(image, cmap='gray')
        ax.imshow(cleared, cmap=cmap, alpha=1.0)  # Overlay the filled mask on the image
    return cleared

### Outputs

In [162]:
def print_regions(regions, figsize=(10, 10), image=None):
    """
    Print the regions as overlay of the original image.
    :param regions: Regions detected from `scikit-learn` `label` function.
    :param figsize: Tuple defining the figsize. Default is (10, 10).
    :param image: Image object representing the original image. Default is None.
    :return: Numpy array with regions represented as binary.
    """
    if image is None:
        raise ValueError("Image must be provided to visualize regions.")
    # Add a white mask on the regions of interest
    region_mask = get_region_mask(regions, image)
    print(f"Number of unique labels found: {len(regions)}")
    colors = [(0, 0, 0, 0), (1, 1, 1, 1)]  # Transparent black, opaque white
    cmap = mcolors.ListedColormap(colors)
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(image, cmap='gray')
    
    ax.imshow(region_mask, cmap=cmap, alpha=1.0) # For the internal regions
    # ax.imshow(combined_mask, cmap=cmap, alpha=1.0) # For the boundary
    return region_mask

In [163]:
def print_region_boundaries(region_properties, image = None, region_threshold = 0, show_mask = False, mask_image = None):
    """
    Print the regions boundaries as overlay of the original image.
    :param region_properties: Regions detected from `scikit-learn` `label` function.
    :param image: Image object representing the original image. Default is None.
    :param region_threshold: Minimum size of the region to be considered.
    :param show_mask: Boolean, whether to show the regions masks.
    :param mask_image: Image object representing the region mask. Default is None.
    :return: None
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    if image is None:
        raise ValueError("Image must be provided to visualize regions.")
    ax.imshow(image, cmap='gray')
    
    if show_mask and mask_image is not None:
        # If a mask image is provided, display it
        cmap = mcolors.ListedColormap([(0, 0, 0, 0), (1, 1, 1, 1)])
        ax.imshow(mask_image, cmap=cmap, alpha=0.5)  # Overlay the mask on the image
    
    for region in region_properties:
        # take regions with large enough areas
        # can adjust
        if region.area >= region_threshold:
            # draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle(
                (minc, minr),
                maxc - minc,
                maxr - minr,
                fill=False,
                edgecolor='red',
                linewidth=2,
            )
            ax.add_patch(rect)
    
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
# visualize all the annotations on the image
def visualize_annotations(image, annotations):
    """
    Visualize the COCO formated segmentations
    :param image: Original image
    :param annotations: Annotation object
    :return: None
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image, cmap='gray')

    for annotation in annotations:
        bbox = annotation['bbox']
        rect = mpatches.Rectangle(
            (bbox[0], bbox[1]),
            bbox[2],
            bbox[3],
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        # Create polygon from segmentation
        segmentation = annotation['segmentation']
        if segmentation:
            for seg in segmentation:
                poly = mpatches.Polygon(np.array(seg).reshape(-1, 2), closed=True, fill=False, edgecolor='blue',
                                        linewidth=1)
                ax.add_patch(poly)

        ax.add_patch(rect)

    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
def mask_to_coco_polygon(mask, image_id, category_id, annotation_id, image_width, image_height):
    """
    Export mask to COCO formatted polygon.
    :param mask: Mask of the ROIs generated from the image.
    :param image_id: ID of the original image.
    :param category_id: Category of the ROIs generated from the image.
    :param annotation_id: ID of the annotation of the ROIs generated from the image.
    :param image_width: Width of the original image.
    :param image_height: Height of the original image.
    :return: Array of polygons in COCO format.
    """
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    segmentations = []
    for contour in contours:
        # Flatten the contour points and convert to list
        segmentation = contour.flatten().tolist()
        segmentations.append(segmentation)

    # Calculate bounding box (x_min, y_min, width, height)
    x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
    bbox = [x, y, w, h]

    # Calculate area
    area = np.sum(mask)

    annotation = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_id,
        "segmentation": segmentations,
        "area": float(area),
        "bbox": bbox,
        "iscrowd": 0
    }
    return annotation

## Loading the Image

In [164]:
# Directory to the scan images
data_path = r'my/data/source'

In [ ]:
#Set image
image = os.path.join(data_path, 'my_image.png')
if not os.path.exists(image):
    raise FileNotFoundError(f"Image not found at {image}")

# Print the image as output
print(f"Image loaded from: {image}")
image = Image.open(image).convert('L')  # Convert to grayscale
# Print the image
print(f"Image size: {image.size}, Mode: {image.mode}")
# Convert to numpy array
image = np.array(image)
# Print the numpy array shape
print(f"Numpy array shape: {image.shape}, Data type: {image.dtype}")
plt.imshow(image, cmap='gray')
plt.figure(figsize=(10, 10))
plt.close()

## Detecting Bounding Boxes

### Create a Binary Mask

In [ ]:
# apply threshold
thresh = threshold_otsu(image)
bw = closing(image > thresh, footprint_rectangle((5, 5), decomposition='sequence'))

# Print the binary image shape
print(f"Binary image shape: {bw.shape}, Data type: {bw.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(bw, cmap='gray')

In [ ]:
# remove artifacts connected to image border
cleared = cleanup_mask(bw, buffer_size=0, print_output=False, image=image, min_object_size=0, min_hole_size=5)
# Print the cleared image shape
print(f"Cleared image shape: {cleared.shape}, Data type: {cleared.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(cleared, cmap='gray')

### Labeling Image Regions

In [ ]:
# label image regions
label_image = label(cleared)

# Print the number of labels found
num_labels = np.max(label_image)
print(f"Number of labels found: {num_labels}")
# Print the labeled image shape
print(f"Labeled image shape: {label_image.shape}, Data type: {label_image.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(label_image, cmap='nipy_spectral')

### Draw Bounding Boxes from Regions

In [ ]:
# Show bounding boxes around the labeled regions.
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0, bg_color=None, kind='overlay')

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image, cmap='gray')

for region in regionprops(label_image):
    # take regions with large enough areas
    # can adjust
    if region.area >= 100:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle(
            (minc, minr),
            maxc - minc,
            maxr - minr,
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

### Dilate the Label Mask

In [ ]:
# Use dilation to connect gaps in the boundary
# dilated_image = cleared.copy()
# Dilate the image for 10 times with a rectangular footprint

dilated_image = morphology.binary_dilation(cleared, footprint_rectangle((3, 3), decomposition='sequence'))
# Print the dilated image shape
print(f"Dilated image shape: {dilated_image.shape}, Data type: {dilated_image.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(dilated_image, cmap='gray')

In [ ]:
# Clear again
cleared = cleanup_mask(dilated_image, buffer_size=0, print_output=False, image=image, min_object_size=5, min_hole_size=10)
# Print the cleared image shape
print(f"Cleared image shape: {cleared.shape}, Data type: {cleared.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(cleared, cmap='gray')

### Labeling Image Regions from Dilated Borders

In [ ]:
# label image regions
label_image = label(cleared)

# Print the number of labels found
num_labels = np.max(label_image)
print(f"Number of labels found: {num_labels}")
# Print the labeled image shape
print(f"Labeled image shape: {label_image.shape}, Data type: {label_image.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(label_image, cmap='nipy_spectral')

### Draw Bounding Boxes from Regions of Dilated Labels

In [ ]:
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0, bg_color=None, kind='overlay')

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image, cmap='gray')

for region in regionprops(label_image):
    # take regions with large enough areas
    # can adjust
    if region.area >= 100:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle(
            (minc, minr),
            maxc - minc,
            maxr - minr,
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

### Create Mask Around ROIs

In [ ]:
# With the label_image, we can see the boundaries of ROIs. We want to create mask with everything inside the boundaries.
# Create a mask for the ROIs
mask = np.zeros_like(label_image, dtype=np.uint8)
# Fill the mask with the labeled regions
for region in regionprops(label_image):
    # if region.area >= 100:  # Adjust the area threshold as needed
    minr, minc, maxr, maxc = region.bbox
    mask[minr:maxr, minc:maxc] = 1  # Fill the mask with 1s for the region

# Print the mask shape
print(f"Mask shape: {mask.shape}, Data type: {mask.dtype}")
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image, cmap='gray')

ax.imshow(mask, cmap='gray', alpha=0.3)  # Overlay the mask on the image
ax.imshow(label_image, cmap='nipy_spectral', alpha=0.3)

# plt.figure(figsize=(10, 10))
# plt.imshow(mask, cmap='gray')

### Try to Fill the Regions
We have the boundary of each of the regions. Let's try to fill the regions to generate complete mask.

#### Use `binary_fill_holes` to Fill the Mask

In [ ]:
# With the label_image, we can see the boundaries of ROIs. We want to create mask with everything inside the boundaries.
# To achieve that we will use fill_holes to fill the empty spaces inside the boundaries.
object_masks = {}
combined_mask = np.zeros_like(label_image, dtype=bool)

for region in regionprops(label_image):
    if region.area < 100:  # Skip small regions
        continue
    region_label = region.label
    min_row, min_col, max_row, max_col = region.bbox

    # Create a blank mask for the entire image
    mask = np.zeros_like(label_image, dtype=bool)

    # Create a slice of the label image and find mask for current label
    region_slice = label_image[min_row:max_row, min_col:max_col]
    rows, cols = np.where(region_slice == region_label)

    # Offset coordinates to fit the main image
    rows += min_row
    cols += min_col

    # Set the mask inside the bounding box to True where label matches
    mask[rows, cols] = True
    
    # Fill holes in the mask
    structure = np.ones((3, 3), dtype=bool) # 3x3 structure works best for filling smaller holes.
    filled_mask = ndimage.binary_fill_holes(mask, structure=structure).astype(bool)

    object_masks[region_label] = filled_mask
    
    # Combine the mask with the main mask
    combined_mask |= filled_mask
    

# Print the mask shape
print(f"Mask shape: {combined_mask.shape}, Data type: {combined_mask.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(combined_mask, cmap='gray')

In [ ]:
print(f"Number of unique labels found: {len(object_masks)}")
colors = [(0, 0, 0, 0), (1, 1, 1, 1)]  # Transparent black, opaque black
cmap = mcolors.ListedColormap(colors)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image, cmap='gray')
for object_label, mask in object_masks.items():
    ax.imshow(mask, cmap=cmap, alpha=1.0)

#### Manually Fill the ROIs
We'll use the custom functions we wrote to manually try to fill the ROIs.

In [ ]:
regions = find_regions_of_interest(image, combined_mask, tolerance=0)
region_mask = print_regions(regions, image=image)

In [ ]:
# It looks like the first method is the cleanest one for finding regions of interest. Let's visualize the mask.
plt.figure(figsize=(10, 10))
plt.imshow(region_mask, cmap='gray')

### Gradually fill the regions
We will call the cleanup function multiple times to gradually fill the regions and see how the mask changes. This should go through a trial and error process to find how many times we need to do this. Going through the process too many times often remove ROIs or does not improve results at all.

In [ ]:
filled_mask = ndimage.binary_fill_holes(region_mask).astype(bool)
plt.figure(figsize=(10, 10))
plt.imshow(filled_mask, cmap='gray')

In [ ]:
cleaned_mask = cleanup_mask(filled_mask, buffer_size=0, print_output=False, image=image, min_object_size=10, min_hole_size=5, try_binary_fill_holes=True)
plt.figure(figsize=(10, 10))
plt.imshow(cleared, cmap='gray')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(cleaned_mask, cmap='gray')
plt.axis('off')

### Create Bounding Boxes from the Filled Mask

In [193]:
# Use the filled mask to create bounding boxes around the sand grains.
label_image = label(cleaned_mask)

In [ ]:
# Print the number of labels found
num_labels = np.max(label_image)
print(f"Number of labels found: {num_labels}")
# Print the labeled image shape
print(f"Labeled image shape: {label_image.shape}, Data type: {label_image.dtype}")
plt.figure(figsize=(10, 10))
plt.imshow(label_image, cmap='nipy_spectral')

In [ ]:
region_properties = regionprops(label_image)
print_region_boundaries(region_properties, image=image, region_threshold=100, show_mask=True, mask_image=cleaned_mask)
print_region_boundaries(region_properties, image=image, region_threshold=100, show_mask=False, mask_image=cleaned_mask)

In [218]:
contours, _ = cv2.findContours(cleaned_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#### Create Segmentation in COCO Format from the Bounding Boxes

In [226]:
segmentations = []
annotations = []
for contour in contours:
    # Flatten the contour points and convert to list
    segmentation = contour.flatten().tolist()
    segmentations.append(segmentation)

    # Calculate bounding box (x_min, y_min, width, height)
    x, y, w, h = cv2.boundingRect(np.array(segmentation).reshape(-1, 2))
    bbox = [x, y, w, h]
    area = int(w) * int(h)
    if area > 10:
        annotation = {
            "id": 1,
            "image_id": 1,
            "category_id": 1,
            "segmentation": segmentations,
            "area": float(area),
            "bbox": bbox,
            "iscrowd": 0
        }
        annotations.append(annotation)

In [ ]:
visualize_annotations(image, annotations)